In [ ]:
import os
import pygplates
import numpy as np
import pandas as pd
import pyvista as pv
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from IPython.display import Video
from sklearn.cluster import DBSCAN
from scipy.interpolate import griddata
from scipy.spatial.transform import Rotation as R

We define some parameters before we begin.

In [ ]:
earthRadius = 6371 #Radius of earth
heightAmplificationFactor = 60 #How much to amplify topography in visualizations

#Properties representing time range and time steps of our simulation
startTime = 10
endTime = 0
deltaTime = 5

#Properties specifying the location of data files
platePolygonsDirectory = './dataPygplates/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies_PMAG.gpmlz'
rotationsDirectory = './dataPygplates/Matthews_etal_GPC_2016_410-0Ma_GK07_PMAG.rot'

# Read Data and Spherical Polar Coordinate Transformations

To initiate our earth, we read a data files containing the Longitude, Latitude, and Elevation of the earth at various geological times. This data is a good approximation of the earth's historical surface, was generated by hand and is a good starting point for our tectonic simulations. These data files were downloaded from the link bellow:
<br> https://www.earthbyte.org/paleodem-resource-scotese-and-wright-2018/

In [ ]:
#Read initial landscape data at specified time from file which is in the form of (lon, lat, height)
def getInitialEarth(time, paleoDemsPath='./PaleoDEMS'):
    
    #Get path of initial landscape data file at specified time
    paleoDemsPath = Path(paleoDemsPath)
    initialLandscapePath = list(paleoDemsPath.glob('**/*%03.fMa.csv'%time))[0]
    
    #Read data and split by newline and commas to create numpy array of data
    initialLandscapeFileLines = open(initialLandscapePath).read().split('\n')[1:-1]
    initLandscapeData = [line.split(',') for line in initialLandscapeFileLines]
    initLandscapeData = np.array(initLandscapeData).astype(float)
    
    #Set heights from metres to kilometers and return data
    initLandscapeData[:, 2] /= 1000
    return initLandscapeData

#Create mesh of the initial data
initData = getInitialEarth(startTime)
initialDataMesh = pv.PolyData(initData).delaunay_2d()

#Create a plotter for visualizing the initial data
initialPlotter = pv.Plotter(notebook=True)
initialPlotter.add_mesh(initialDataMesh, scalars=initData[:, 2])
initialPlotter.camera_position = 'xy'
initialPlotter.camera.position = (0, 0, 350.0)
initialPlotter.show(jupyter_backend='panel', window_size=[800, 400])

The earth here seems a bit flat, but apparently its meant to be a sphere. To transform a coordinates from cartesian (XYZ) to spherical polar coordinates $(r, \theta, \phi)$ we define a few coordinate transformation functions. Our coordinate transformations are given by


$r = \sqrt{x^2 + y^2 + z^2}$ <br>
$\theta = \tan^{-1} (\frac{y}{x})$ <br>
$\phi = \cos^{-1} (\frac{z}{r})$

and to convert back to cartesian coordinates:

$x = r \cos \theta \sin \phi$ <br> 
$y = r \sin \theta \sin \phi$ <br>
$z = r \cos \phi$

By default, these functions will take angles in the form of degrees, which is suitable for longitudinal and latitudinal coordinates, however they can also accept angles in the form of degrees by setting *useLonLat* to False. For more details about spherical polar coordinate transformations, see the article linked bellow: <br>
https://mathworld.wolfram.com/SphericalCoordinates.html

<div>
<img src="files/Images/SphericalTransforms.jpg" width="600">
</div>

In [ ]:
#Coordinate transformation from spherical polar to cartesian
def polarToCartesian(radius, theta, phi, useLonLat=True):
    if useLonLat == True:
        theta, phi = np.radians(theta+180.), np.radians(90. - phi)
    X = radius * np.cos(theta) * np.sin(phi)
    Y = radius * np.sin(theta) * np.sin(phi)
    Z = radius * np.cos(phi)
    
    #Return data either as a list of XYZ coordinates or as a single XYZ coordinate
    if (type(X) == np.ndarray):
        return np.stack((X, Y, Z), axis=1)
    else:
        return np.array([X, Y, Z])

#Coordinate transformation from cartesian to polar
def cartesianToPolarCoords(XYZ, useLonLat=True):
    X, Y, Z = XYZ[:, 0], XYZ[:, 1], XYZ[:, 2]
    R = (X**2 + Y**2 + Z**2)**0.5
    theta = np.arctan2(Y, X)
    phi = np.arccos(Z / R)
    
    #Return results either in spherical polar or leave it in radians
    if useLonLat == True:
        theta, phi = np.degrees(theta), np.degrees(phi)
        lon, lat = theta - 180, 90 - phi
        lon[lon < -180] = lon[lon < -180] + 360
        return R, lon, lat
    else:
        return R, theta, phi

Now that we have set up our coordinate transformation functions, we use those to plot the earth data on a sphere. To make the landscape more visible, we exegerate the heights topology.

In [ ]:
#Exagerate the heights
lon, lat, height = initData[:, 0], initData[:, 1], initData[:, 2]
exageratedRadius = height * heightAmplificationFactor + earthRadius

#Create XYZ coordinates from polar coordinates
initialEarthXYZ = polarToCartesian(exageratedRadius, lon, lat)

#Create mesh with same faces as the flat initialDataMesh
earthFaces = initialDataMesh.faces
initialEarthMesh = pv.PolyData(initialEarthXYZ, earthFaces)

#Create a plotter for visualizing the round earth
sphereEarthPlotter = pv.PlotterITK()
sphereEarthPlotter.add_mesh(initialEarthMesh, scalars=height)
sphereEarthPlotter.show()

# Moving Tectonic Plates

To specify which tectonic plate a particular vertex on our sphere belongs to, we create a list of *Plate Ids* where each vertex is given a number based on which plate they belong to. To move tectonic plates, we create a rotation quaternion for each plate, and apply rotations to all vertices based on their plate ids.

We use the library *pygplates* to assign plate ids to vertices. *Pygplates* requires a list of point features for each vertex on our sphere and assigns plate ids to those.

In [ ]:
#Creates a list of point features for each vertex on our sphere
def createPointFeatures(lon, lat):
    pointsOnSphere = [pygplates.PointOnSphere(float(lat[i]), float(lon[i])) for i in range(len(lon))]
    pointFeatures = []
    for point in pointsOnSphere:
        pointFeature = pygplates.Feature()
        pointFeature.set_geometry(point)
        pointFeatures.append(pointFeature)
    return pointFeatures

#Returns a list of plate Ids for points on our sphere
def createPlateIdsAtTime(time, pointFeatures, 
        platePolygonsDirectory = './dataPygplates/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies_PMAG.gpmlz',
        rotationsDirectory = './dataPygplates/Matthews_etal_GPC_2016_410-0Ma_GK07_PMAG.rot'):
    
    assignedPointFeatures = pygplates.partition_into_plates(
        platePolygonsDirectory,
        rotationsDirectory,
        pointFeatures,
        reconstruction_time=float(time),
        properties_to_copy = [
            pygplates.PartitionProperty.reconstruction_plate_id,
            pygplates.PartitionProperty.valid_time_period])
    featureIds = [feat.get_reconstruction_plate_id() for feat in assignedPointFeatures]
    return np.array(featureIds)

#Run the new functions to get plate ids
pointFeatures = createPointFeatures(lon, lat)
plateIds = createPlateIdsAtTime(startTime, pointFeatures)

#Create a plot of the earth with vertices color coded by plate ids
plateIdsPlotter = pv.PlotterITK()
plateIdsPlotter.add_mesh(initialEarthMesh, scalars=plateIds)
plateIdsPlotter.show()

To move the tectonic plates, we use *pygplates* to get an axis $\textbf{u}$ and angle $\theta$ of rotation. We use those to create a rotation quaternion, and apply the quaternion to all vertices that belong on a particular tectonic plate.

A rotation quaternion is given by:

$\cos \frac{\theta}{2} + (u_x \hat{\textbf{i}} + u_y \hat{\textbf{j}} + u_z \hat{\textbf{k}}) \sin \frac{\theta}{2}$ 

where $\hat{\textbf{i}}, \hat{\textbf{j}}$ and  $\hat{\textbf{k}}$ are unit vectors representing cartesian axis. To apply the rotations, we use the *scipy.spatial.Rotation* library.

In [ ]:
#Returns a rotation quaternion
def quaternion(axis, angle):
    return [np.sin(angle/2) * axis[0], 
            np.sin(angle/2) * axis[1], 
            np.sin(angle/2) * axis[2], 
            np.cos(angle/2)]

#Get stage rotation data from pygplates and return a scipy rotation
def getRotations(time, deltaTime, plateIds):
    rotations = {}
    for plateId in np.unique(plateIds):
        stageRotation = rotationModel.get_rotation(int(time-deltaTime), int(plateId), int(time))
        stageRotation = stageRotation.get_euler_pole_and_angle()

        #Create rotation quaternion from axis and angle
        axisLatLon = stageRotation[0].to_lat_lon()
        axis = polarToCartesian(1, axisLatLon[1], axisLatLon[0])
        angle = stageRotation[1]
        rotations[plateId] = R.from_quat(quaternion(axis, angle))
    return rotations

#Move tectonic plates along the sphere by applying rotations to vertices with appropriate plate ids
def movePlates(sphereXYZ, plateIds, rotations):
    newXYZ = np.copy(sphereXYZ)
    for idx in np.unique(plateIds):
        rot = rotations[idx]
        newXYZ[plateIds == idx] = rot.apply(newXYZ[plateIds == idx])
    return newXYZ

#Create mesh with moved tectonic plates
rotationModel = pygplates.RotationModel(rotationsDirectory)
rotations = getRotations(startTime, deltaTime, plateIds)
earthWithMovedPlatesXYZ = movePlates(initialEarthXYZ, plateIds, rotations)
earthWithMovedPlatesMesh = pv.PolyData(earthWithMovedPlatesXYZ, earthFaces)

#Create an earth plot with the meshe's edges displayed
plateIdsPlotter = pv.Plotter(notebook=True)
plateIdsPlotter.add_mesh(earthWithMovedPlatesMesh, scalars=plateIds, show_edges=True)
plateIdsPlotter.show(jupyter_backend='panel', window_size=[800, 400])

It is worth mentioning that the *createPlateIdsAtTime()* function above is very slow. After measuring it's execution time, it took about 10 seconds to create a list of plateIds, compared to 1 second of doing everything else in the *runTectonicSimulation()* function from the finished code bellow.

To speed things up, we will save the plateIds as a file each time we run *createPlateIdsAtTime()*, and read the data from file if it already exists.

In [ ]:
def getPlateIdsAtTime(time, lon, lat, 
        platePolygonsDirectory = './dataPygplates/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies_PMAG.gpmlz',
        rotationsDirectory = './dataPygplates/Matthews_etal_GPC_2016_410-0Ma_GK07_PMAG.rot',
        plateIdsDirectory='PlateIdData'):
    
    #Create data folder if it doesn't already exists
    if not os.path.isdir(plateIdsDirectory):
        os.mkdir('./' + plateIdsDirectory)
    
    #Create a file name, and read from file if it already exists
    fileName = './{}/time{}size{}.txt'.format(plateIdsDirectory, time, len(lon))
    if os.path.exists(fileName):
        plateIds = pd.read_csv(fileName, header=None)
        plateIds = np.array(plateIds)[:, 0]
    
    #Otherwise calculate plate ids and write to file
    else:
        #Calculate plate ids
        print('Creating new plate ID file')
        pointFeatures = createPointFeatures(lon, lat)
        plateIds = createPlateIdsAtTime(time, 
                        pointFeatures, 
                        platePolygonsDirectory = platePolygonsDirectory, 
                        rotationsDirectory = rotationsDirectory)
        
        #Write to file
        with open(fileName, 'w') as file:
            for idx in plateIds:
                file.write('{}\n'.format(idx))
    return plateIds

# Remeshing the Sphere

By moving tectonic plates, vertices around plate boundaries may overlap when plates converge or they may move too far from each other when plates diverge. To fix this we will interpolated the heights of the new sphere back onto the polar coordinates of the old sphere.

In [ ]:
#Create template coordinates to interpolate with
templateLonLat = np.stack((initData[:, 0], initData[:, 1]), axis=1)

#Create reference coordinates of the sphere with moved plates to interpolate from
movedLonLat = cartesianToPolarCoords(earthWithMovedPlatesXYZ)
movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)
heights = initData[:, 2]

#Remesh the sphere using interpolation
interpolatedHeights = griddata(movedLonLat, heights, templateLonLat)
whereNAN = np.argwhere(np.isnan(interpolatedHeights))
interpolatedHeights[whereNAN] = griddata(movedLonLat, heights, templateLonLat[whereNAN], method='nearest')

#Create new moved sphere with a better mesh
interpolatedRadius = interpolatedHeights * heightAmplificationFactor + earthRadius
interpolatedSphereXYZ = polarToCartesian(interpolatedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
interpolatedSphereXYZ = pv.PolyData(interpolatedSphereXYZ, earthFaces)

#Create plot of the remesh
plateIdsPlotter = pv.Plotter(notebook=True)
plateIdsPlotter.add_mesh(interpolatedSphereXYZ, scalars=interpolatedHeights)
plateIdsPlotter.camera_position = 'xz'
plateIdsPlotter.camera.azimuth = 200
plateIdsPlotter.show(jupyter_backend='panel', window_size=[800, 400])

Hmm... The mountain ranges of South America on the above earth dissappeared!!! What happened? The problem is that the interpolation scheme sampled heights of subducting vertices, but we don't want that. We only want to sample points of the overiding plates.

To fix this issue, we can set heights of subducting vertices to the heights of nearby over-riding vertices. But to do this, we first have to identify vertices that are in the subduction region. To identify these, we transform our coordinates into a cylinder so that vertices **not** on plate boundaries are equally spaced apart. Vertices in the subduction regions will be closer to each other and can therefore be identified by the DBSCAN clustering algorithm. Then, for each vertex belonging to a cluster, we set its height to the maximum of its nearest neighbours.

With $\theta$ and $z$ representing lon/lat coordinates (a flat earth), we use the following coordinate transformation to create a cylinder:

$$
\begin{align}
x & = r \cos \theta \\
y & = r \sin \theta \\
z & = z
\end{align}
$$

We want to chose a radius $r$ such that the vertices form a grid with equal spacing along both lon/lat directions. Let $m$ be the resolution of vertices along the longitudinal direction, and $n$ be the resolution along the latitudinal direction. The circumference of the cylinder is $2 \pi r$, so the spacing along the longitudinal direction will be $\frac{2 \pi r}{m}$. Let $h$ be the distance from the north to south pole, the spacing along the latitudinal direction is $\frac{h}{n}$. We want both spacings to be equal, so our desired cylinder radius is:

$$
\begin{align}
\large \frac{2 \pi r}{m} & = \large \frac{h}{n} \\
\large r & = \large \frac{m h}{2 \pi n}
\end{align}
$$

Within this new cylindrical coordinate system, we can now use the DBSCAN clustering algorithm to identify overlapping plate vertices. Given a threshold distance, the DBSCAN algorithm groups vertices together which are closer than the specified distance. Here we chose a threshold distance that is a bit shorter than the normal grid spacing of the vertices:

We provide an example bellow, where overlapping vertices are shown in yellow:

In [ ]:
#Coordinate transformation functions from cartesian to cylindrical polar coordinates
def cartesianToCylindrical(X, Y, Z):
    r = (X**2 + Y**2)**0.5
    theta = np.arctan2(Y, X)
    return np.stack((r, theta, Z), axis=1)

#Coordinate transformation functions from cylindrical polar coordinates to cartesian
def cylindricalToCartesian(r, theta, Z, useDegrees=True):
    if useDegrees == True:
        theta = np.radians(theta+180.)
    X = r * np.cos(theta)
    Y = r * np.sin(theta)
    return np.stack((X, Y, Z), axis=1)

maxClusterSize = 3

#Get number of subdivisions along lon/lat axii
m = len(np.unique(initData[:, 0])) - 1
n = len(np.unique(initData[:, 1])) - 1
h = np.max(movedLonLat[:, 1]) - np.min(movedLonLat[:, 1])

#Create clyinder
cylinderRadius = m * h / (np.pi * n * 2)
cylinderXYZ = cylindricalToCartesian(cylinderRadius, movedLonLat[:, 0], movedLonLat[:, 1])
cylinderMesh = pv.PolyData(cylinderXYZ)

#Run the clustering algorithm
spacing = 300 / m
cluster = DBSCAN(eps=spacing, min_samples=maxClusterSize, n_jobs=4).fit(cylinderXYZ)
isCluster = (cluster.labels_ != -1)

#Points near the poles are clustered, even when they are not plate boundaries, so we manually set those to False
isCluster[np.abs(initData[:, 1])>80] = False

#Show results
pl = pv.Plotter(notebook=True)
pl.add_mesh(pv.Cylinder(radius=cylinderRadius-0.5, height=180, direction=(0, 0, 1)))
pl.add_mesh(cylinderMesh, scalars=isCluster)
pl.show(jupyter_backend='panel', window_size=[800, 400])

Now that we have identified which vertices bellong to converging plate boundaries, we set those vertices to the maximum of their nearest neighbours. We do this in the code bellow, and visualize results by showing the new heights in yellow, and old heights in red. Feel free to pan the viewport camera to see results more clearly.

In [ ]:
numOfNeighbsToConsider = 6

#Create KDTree to find nearest neighbours of each point in cluster
pointsInClusterLonLat = cylinderXYZ[isCluster]
clusterKDTree = cKDTree(pointsInClusterLonLat).query(pointsInClusterLonLat, k=numOfNeighbsToConsider+1)

#Get heights of nearest neighbours
heightsInCluster = heights[isCluster]
clusterPointsNeighboursId = clusterKDTree[1]
neighbourHeights = heightsInCluster[clusterPointsNeighboursId[:, 1:]]

#For points in cluster, set new heights to the maximum height of nearest neighbours
newHeights = np.copy(heights)
newHeights[isCluster] = np.max(neighbourHeights, axis=1)

#Create meshes for plot
flatEarthXYZ = np.stack((movedLonLat[:, 0], movedLonLat[:, 1], heights), axis=1)
newFlatEarthXYZ = np.stack((movedLonLat[:, 0], movedLonLat[:, 1], newHeights), axis=1)

#Show results
pl = pv.Plotter(notebook=True)
pl.add_mesh(flatEarthXYZ[isCluster], color='red')
pl.add_mesh(newFlatEarthXYZ, scalars=isCluster)
pl.camera_position = 'xy'
pl.camera.position = (0, 0, 350.0)
pl.show(jupyter_backend='panel', window_size=[800, 400])

Now that we have demonstrated how the remesh algorithm works, we create functions of the algorithm. We provide an example of the final remesh, but this time with **no** disappearing mountain ranges. Note that although the mountain ranges are a bit larger than before, this is preferable since mountains grow during the subduction process anyways. This is also less of an issue when our earth mesh has a higher resolution.

In [ ]:
thetaResolution = len(np.unique(initData[:, 0])) - 1
phiResolution = len(np.unique(initData[:, 1])) - 1
numOfNeighbsToConsider = 6
clusterThresholdProportion = 300 / 360
maxClusterSize = 3

def getHeightsForRemesh(movedLonLat, heights):
    m = thetaResolution
    n = phiResolution
    numOfNeighbs = numOfNeighbsToConsider
    h = np.max(movedLonLat[:, 1]) - np.min(movedLonLat[:, 1])
    
    #Create clyinder
    cylinderRadius = m * h / (np.pi * n * 2)
    cylinderXYZ = cylindricalToCartesian(cylinderRadius, movedLonLat[:, 0], movedLonLat[:, 1])

    #Run the clustering algorithm
    threshHoldDist = clusterThresholdProportion * 360 / m
    cluster = DBSCAN(eps=threshHoldDist, min_samples=maxClusterSize).fit(cylinderXYZ)
    isCluster = (cluster.labels_ != -1)
    
    #Create KDTree to find nearest neighbours of each point in cluster
    pointsInClusterLonLat = cylinderXYZ[isCluster]
    clusterKDTree = cKDTree(pointsInClusterLonLat).query(pointsInClusterLonLat, k=numOfNeighbs+1)
    
    #Get heights of nearest neighbours
    heightsInCluster = heights[isCluster]
    clusterPointsNeighboursId = clusterKDTree[1]
    neighbourHeights = heightsInCluster[clusterPointsNeighboursId[:, 1:]]

    #For points in cluster, set new heights to the maximum height of nearest neighbours
    newHeights = np.copy(heights)
    newHeights[isCluster] = np.max(neighbourHeights, axis=1)
    return newHeights

def remeshSphere(templateLonLat, movedLonLat, heights):
    heightsForRemesh = getHeightsForRemesh(movedLonLat, heights)
    newHeights = griddata(movedLonLat, heightsForRemesh, templateLonLat)
    whereNAN = np.argwhere(np.isnan(newHeights))
    newHeights[whereNAN] = griddata(movedLonLat, heightsForRemesh, templateLonLat[whereNAN], method='nearest')
    return newHeights

#Create template coordinates to interpolate with
templateLonLat = np.stack((initData[:, 0], initData[:, 1]), axis=1)

#Create reference coordinates of the sphere with moved plates to interpolate from
movedLonLat = cartesianToPolarCoords(earthWithMovedPlatesXYZ)
movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)
heights = initData[:, 2]

#Remesh the sphere using interpolation
interpolatedHeights = remeshSphere(templateLonLat, movedLonLat, heights)

#Create new moved sphere with the better mesh
interpolatedRadius = interpolatedHeights * heightAmplificationFactor + earthRadius
interpolatedSphereXYZ = polarToCartesian(interpolatedRadius, templateLonLat[:, 0], templateLonLat[:, 1])
interpolatedSphereXYZ = pv.PolyData(interpolatedSphereXYZ, earthFaces)

#Show results
plateIdsPlotter = pv.Plotter(notebook=True)
plateIdsPlotter.add_mesh(interpolatedSphereXYZ, scalars=interpolatedHeights)
plateIdsPlotter.camera_position = 'xz'
plateIdsPlotter.camera.azimuth = 200
plateIdsPlotter.show(jupyter_backend='panel', window_size=[800, 400])

# Putting it All Together

Now that we have gone through the main methodology in moving tectonic plates along a sphere, we create a class. The advantage of using a class is that all data is stored within the class, and we can run and control the simulation using only a few functions. We also create a method animating the results.

In [ ]:
class Earth:
    def __init__(self,
                 startTime = 10,
                 endTime = 0,
                 deltaTime = 5,
                 earthRadius = 6378.137,
                 heightAmplificationFactor = 60,
                 platePolygonsDirectory = './dataPygplates/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies_PMAG.gpmlz',
                 rotationsDirectory = './dataPygplates/Matthews_etal_GPC_2016_410-0Ma_GK07_PMAG.rot',
                 movieOutputDir = 'TectonicSimulation.mp4',
                 animationFramesPerIteration = 8,
                 numOfNeighbsForRemesh = 6,
                 clusterThresholdProportion = 300 / 360
                ):
        
        #Set attribute from class initialization
        self.startTime = startTime
        self.endTime = endTime
        self.deltaTime = deltaTime
        self.earthRadius = earthRadius
        self.heightAmplificationFactor = heightAmplificationFactor
        self.platePolygonsDirectory = platePolygonsDirectory
        self.rotationsDirectory = rotationsDirectory
        self.movieOutputDir = movieOutputDir
        self.animationFramesPerIteration = animationFramesPerIteration
        self.numOfNeighbsForRemesh = numOfNeighbsForRemesh
        self.clusterThresholdProportion = clusterThresholdProportion
        
        #Pre-calculate commonly used attributes
        initData = getInitialEarth(self.startTime)
        self.lon = initData[:, 0]
        self.lat = initData[:, 1]
        self.lonLat = np.stack((initData[:, 0], initData[:, 1]), axis=1)
        self.sphereXYZ = polarToCartesian(1, initData[:, 0], initData[:, 1])
        self.heightHistory = [initData[:, 2]]
        self.simulationTimes = np.arange(self.startTime, self.endTime-self.deltaTime, -self.deltaTime)
        self.rotationModel = pygplates.RotationModel(self.rotationsDirectory)
        self.pointFeatures = createPointFeatures(initData[:, 0], initData[:, 1])
        self.earthFaces = pv.PolyData(initData).delaunay_2d().faces
        self.thetaResolution = len(np.unique(initData[:, 0])) - 1
        self.phiResolution = len(np.unique(initData[:, 1])) - 1
        self.totalIterations = 1
    
    #rotations = getRotations(startTime, deltaTime, plateIds)
    #earthWithMovedPlatesXYZ = movePlates(initialEarthXYZ, plateIds, rotations)
    
    #Run the simulation at specified times and append results to heightHistory
    def runTectonicSimulation(self):
        for time in self.simulationTimes:
            self.totalIterations += 1
            plateIds = getPlateIdsAtTime(time, self.lon, self.lat)
            rotations = getRotations(time, self.deltaTime, plateIds)
            movedEarthXYZ = movePlates(self.sphereXYZ, plateIds, rotations)
            movedLonLat = cartesianToPolarCoords(movedEarthXYZ)
            movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)
            heights = remeshSphere(self.lonLat, movedLonLat, self.heightHistory[-1])
            self.heightHistory.append(heights)
    
    #Get XYZ coordinates of earth at specified iteration (the default iteration is the latest iteration)
    def getEarthXYZ(self, iteration=-1):
        amplifier = self.heightAmplificationFactor
        lon, lat = self.lonLat[:, 0], self.lonLat[:, 1]
        exageratedRadius = self.heightHistory[iteration] * amplifier + self.earthRadius
        earthXYZ = polarToCartesian(exageratedRadius, lon, lat)
        return earthXYZ
    
    #Create a plot of earth suitable for jupyter notebook at specified iteration
    def showEarth(self, iteration=-1):
        earthXYZ = self.getEarthXYZ(iteration=iteration)
        earthMesh = pv.PolyData(earthXYZ, self.earthFaces)
        plotter = pv.PlotterITK()
        plotter.add_mesh(earthMesh, scalars=self.heightHistory[iteration])
        plotter.show(window_size=[800, 400])
    
    #Create an animation of the earth which is saved as an mp4 file in the current directory
    def animate(self):
        earthXYZ = self.getEarthXYZ(iteration=0)
        earthMesh = pv.PolyData(earthXYZ, self.earthFaces)
        
        #Set up plotter for animation
        plotter = pv.Plotter()
        plotter.add_mesh(earthMesh, scalars=self.heightHistory[0], cmap='gist_earth')
        plotter.show(auto_close=False, window_size=[800, 608])
        plotter.open_movie(self.movieOutputDir)
        plotter.write_frame()
        
        #Draw frames of simulation
        for i in range(self.totalIterations-1):
            earthXYZ = self.getEarthXYZ(iteration=i+1)
            plotter.update_coordinates(earthXYZ, mesh=earthMesh)
            plotter.update_scalars(self.heightHistory[i+1], render=False, mesh=earthMesh)
            for i in range(self.animationFramesPerIteration):
                plotter.write_frame()
        plotter.close()

As shown bellow, running the earth simulation can now be done with just a few simple lines of code.

In [ ]:
earth = Earth()
earth.runTectonicSimulation()

We can easily show the earths state at any iteration of the simulation.

In [ ]:
earth.showEarth(iteration=0)

And we can easily create an animation of the earth which is stored as an mp4 file in this notebook's directory.

In [ ]:
earth.animate()

All functions and classes defined in this notebook can also be found in the *CodeAfterFirstNotebook.py* python script, with perhaps some minor changes. This script will be used in future notebooks to avoid having to redefine everything. The script can be imported and used as follows:

In [1]:
import CodeAfterFirstNotebook as initEarth

#Run the simulation and display results
earth = initEarth.Earth(startTime=30)
earth.runTectonicSimulation()
earth.showEarth(iteration=0)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
#Create an animation of the results
earth.animate()

Note that at this stage, the remeshing algorithm is the slowest part of the simulation. Within the *runSimulation* loop, it takes about 1.8 seconds to complete, meanwhile all other parts of the loop require less than 0.1 seconds to complete. For now, I can't think of ways to speed this up though. We can't use *numba* to, since the algorithm makes use of too many *scipy* functions.